In [2]:
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import col,when,size,expr
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark import sql


In [3]:

conf = SparkConf()
sc = SparkContext(conf=conf)
sqlContext = sql.SQLContext(sc)
spark = SparkSession.builder.appName("stuff").master("local").enableHiveSupport().getOrCreate()
spark

In [4]:
# timestamp is number of seconds since 1970-01-01 
timestamp = 1545730073

# convert the timestamp to a datetime object in the local timezone
dt_object = datetime.fromtimestamp(timestamp)

# print the datetime object and its type
print("dt_object =", dt_object)
print("type(dt_object) =", type(dt_object))

dt_object = 2018-12-25 16:27:53
type(dt_object) = <class 'datetime.datetime'>


In [5]:
c = spark.read.json("raw_json/raw_json/*")


In [6]:
c.printSchema()

root
 |-- GraphImages: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- __typename: string (nullable = true)
 |    |    |-- comments: struct (nullable = true)
 |    |    |    |-- data: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- created_at: long (nullable = true)
 |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |-- owner: struct (nullable = true)
 |    |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |    |-- profile_pic_url: string (nullable = true)
 |    |    |    |    |    |    |-- username: string (nullable = true)
 |    |    |    |    |    |-- text: string (nullable = true)
 |    |    |-- comments_disabled: boolean (nullable = true)
 |    |    |-- dimensions: struct (nullable = true)
 |    |    |    |-- height: long (nullable = true)
 |    |    |    |-- width: long (nullable = true)
 |    |    |-- disp

In [7]:
op = c.select(col("crawler_target.specific_resource_type").alias("soc_med"), size(col("comments.data")).alias("comments_cnt"), col("comments.data.created_time").alias("times"))\
    .withColumn("dates", expr('transform(times, x -> split(x,"T")[0])'))\
    .where((col("GraphImages").isNull()) \
        & (col("crawler_target").isNotNull())\
            & (col("comments.data").isNotNull())\
            )\
    .groupBy(col("soc_med")).count()
op.show(20, False)

+--------+-----+
|soc_med |count|
+--------+-----+
|facebook|751  |
+--------+-----+



In [9]:
op = c.select(col("crawler_target.specific_resource_type").alias("a")).groupBy(col("a")).count()
op.show(20, False)

+---------+------+
|a        |count |
+---------+------+
|instagram|6424  |
|twitter  |172541|
|null     |67589 |
|youtube  |4708  |
|facebook |751   |
+---------+------+



In [12]:
op = c.select(size(col("comments.data")).alias("comments_cnt"))\
    .where((col("GraphImages").isNull()) \
        & (col("crawler_target").isNotNull()) \
        & (c.comments.data.isNotNull())
        )
op.show()

+------------+
|comments_cnt|
+------------+
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
|          25|
+------------+
only showing top 20 rows



In [13]:
op = c.select(col("comments.data.created_time").alias("times"))\
    .withColumn("dates", expr('transform(times, x -> split(x,"T")[0])'))\
    .where((col("GraphImages").isNull()) \
        & (col("crawler_target").isNotNull()) \
        & (c.comments.data.isNotNull())
        )
op.show()

+--------------------+--------------------+
|               times|               dates|
+--------------------+--------------------+
|[2022-01-01T15:32...|[2022-01-01, 2022...|
|[2022-01-01T15:32...|[2022-01-01, 2022...|
|[2022-01-01T01:43...|[2022-01-01, 2022...|
|[2021-12-31T10:14...|[2021-12-31, 2021...|
|[2021-12-29T10:45...|[2021-12-29, 2021...|
|[2021-12-28T10:46...|[2021-12-28, 2021...|
|[2021-12-28T09:17...|[2021-12-28, 2021...|
|[2021-12-28T07:33...|[2021-12-28, 2021...|
|[2021-12-27T13:19...|[2021-12-27, 2021...|
|[2021-12-27T12:45...|[2021-12-27, 2021...|
|[2021-12-27T08:22...|[2021-12-27, 2021...|
|[2021-12-27T08:15...|[2021-12-27, 2021...|
|[2021-12-27T02:18...|[2021-12-27, 2021...|
|[2021-12-25T01:08...|[2021-12-25, 2021...|
|[2021-12-24T08:04...|[2021-12-24, 2021...|
|[2021-12-23T06:45...|[2021-12-23, 2021...|
|[2021-12-22T07:47...|[2021-12-22, 2021...|
|[2021-12-22T05:27...|[2021-12-22, 2021...|
|[2021-12-22T04:45...|[2021-12-22, 2021...|
|[2021-12-21T13:49...|[2021-12-2